In [2]:
import pandas as pd

In [14]:
df = pd.read_csv("spotify_millsongdata.csv")

In [15]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [16]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [17]:
df.shape

(57650, 4)

In [18]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [19]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [20]:
df.head(10)

,artist,song,text
0,Vince Gill,Tryin' To Get Over You,You could have given me a million reasons why ...
1,America,Old Virginia,"Hey, honey, won't you sit down \r\nAnd stop h..."
2,Insane Clown Posse,Red Neck Hoe,"Red Neck Hoe \r\n""Rapping to this bitch with ..."
3,Josh Groban,Anthem,"No man, no madness \r\nThough their sad power..."
4,Hillsong United,I Will Sing,For You have set me free \r\nYou're all that ...
5,Kim Wilde,Just A Feeling,Stand at the mirror and look at my face \r\nS...
6,Alison Krauss,Somewhere In The Vicinity Of The Heart,Thousand miles \r\nOf lonesome highway \r\nD...
7,Divine,Fairy Tales,1 Love ain't like no fairy tale \r\n \r\nIt'...
8,Jimi Hendrix,Bold As Love,"Anger, he smiles, \r\nTowering in shiny metal..."
9,Kate Bush,The King Is Dead,The king is dead \r\nNo doubt about it \r\nW...


In [21]:
df['text'][0]

"You could have given me a million reasons why  \r\nBut it wouldn't change a thing  \r\nYou said it all when you said goodbye  \r\nAnd you took off your wedding ring  \r\n  \r\nI've been tryin' to get over you  \r\nI've been spending time alone  \r\nI've been tryin' to get over you  \r\nBut it'll take dyin' to get it done  \r\n  \r\nAll my friends keep tryin' to fix me up  \r\nThey say I need somebody new  \r\nWhen it comes to love, I've all but given up  \r\n'Cause life don't mean nothin' without you  \r\n  \r\nI've been tryin' to get over you  \r\nI've been spending time alone  \r\nI've been tryin' to get over you  \r\nBut it'll take dyin' to get it done\r\n\r\n"

In [22]:
# df = df.sample(5000)

In [23]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [32]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [31]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

df['text'] = df['text'].apply(lambda x: tokenization(x))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Lenovo/nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\share\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


NameError: name 'tokenization' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [ ]:
similarity[0]

In [ ]:
df[df['song'] == 'Crying Over You']

In [ ]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [ ]:
recommendation('Crying Over You')

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))